### Import Packages

In [ ]:
# Imports data manipulation and visualization packages
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
import pickle

# Imports ML packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.models import model_from_json
from keras.models import load_model
import sklearn.metrics as metrics

### Global Variables

In [ ]:
# Directory where all the images are stored
DATADIR = "Images" 

# The name of each classification. These MUST be the same
# as the name of the folder in which each classified image is stored
CATEGORIES = ["De La Riva", "Fifty50", "Single Leg X"]

# The size of the images that your neural network will use
IMG_SIZE = 50 

### Loads the images and creates training data

In [ ]:
# Creates an array of our images with their classification
for category in CATEGORIES :
	path = os.path.join(DATADIR, category)
	for img in os.listdir(path):
		img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)

# Array for training data
training_data = []

# Create training data
def create_training_data():
	for category in CATEGORIES :
		path = os.path.join(DATADIR, category)
		class_num = CATEGORIES.index(category)
		for img in os.listdir(path):
			try :
				img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
				new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
				training_data.append([new_array, class_num])
			except Exception as e:
				pass

create_training_data()
random.shuffle(training_data)

### Creates 'X' and 'y' files that contain classification information

In [ ]:
X = [] # Features
y = [] # Labels

for features, label in training_data:
	X.append(features)
	y.append(label)

# Reshape our array so it matches 'tensors' of our model
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

# Creating the files containing all the information about your model
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

### Create the model

In [ ]:
# Opening the data files
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

# Normalizing the data (a pixel goes from 0 to 255)
X = X/255.0

# Building the model
model = Sequential()

# We need 3 convolutional layers
model.add(Conv2D(32, (3, 3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# We need 3 hidden layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(128))
model.add(Activation("relu"))

# The output layer with 3 neurons, for 3 classes
model.add(Dense(3))
model.add(Activation("softmax"))

### Compile, fit, and save the model

In [ ]:
# Compiling the model using some basic parameters
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Reshape arrays
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

In [ ]:
# Training the model, with 50 iterations
# validation_split corresponds to the percentage of images used for the validation phase compared to all the images
# In this case, 10% of the images are used to verify the models accuract

history = model.fit(X, y, batch_size = 32, epochs = 40, validation_split = 0.15)

In [ ]:
# Saving the model
model_json = model.to_json()
with open("model.json", "w") as json_file :
	json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")

model_name = 'Guards.model'
model.save(model_name)

### Data Visualization

In [ ]:
# Printing a graph showing the accuracy changes during the training phase
print(history.history.keys())
plt.figure(1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

### File sanitization function and load the model

In [ ]:
# Prepare the input file (image)
def prepare(file):
    IMG_SIZE = 50
    img_array = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img_array = img_array/255.0
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
# Load Model
model = tf.keras.models.load_model(model_name)

## User Input

In [ ]:
# Define what image you want to load and prepare it
input_image = "1.jpg"
image = prepare(input_image)

# Use the model to predict what the image is
prediction = model.predict(image)
prediction = list(prediction[0])
print(CATEGORIES[prediction.index(max(prediction))])